In [1]:
import os
import numpy as np
import math
from annoy import AnnoyIndex
from tqdm import tqdm
import librosa
import pickle

In [2]:
def signal_energy(signal):
    energy = np.sum(np.square(signal))
    return energy

In [3]:
# Hàm tính độ biến đổi (variance) của tín hiệu
def signal_variance(signal):
    mean = np.mean(signal)
    variance = np.sum(np.square(signal - mean))
    return variance

In [4]:
# Hàm tính Zero Crossing Rate (ZCR)
def zero_crossing_rate(signal):
    zcr = 0
    for i in range(1, len(signal)):
        if (signal[i-1] >= 0 and signal[i] < 0) or (signal[i-1] < 0 and signal[i] >= 0):
            zcr += 1
    return zcr / len(signal)

In [5]:
# Hàm tính tần số trung bình (Average Frequency)
def average_frequency(signal, sample_rate):
    fft_result = np.fft.fft(signal)
    fft_freqs = np.fft.fftfreq(len(signal), 1/sample_rate)
    positive_freqs = fft_freqs[:len(fft_freqs)//2]
    amplitudes = np.abs(fft_result[:len(fft_result)//2])
    average_freq = np.sum(positive_freqs * amplitudes) / np.sum(amplitudes)
    return average_freq

In [6]:
def extract_features(target_signal):
    
    sample_rate = 44100  

    # Tính toán các thuộc tính của tín hiệu âm thanh
    energy = signal_energy(target_signal)
    variance = signal_variance(target_signal)
    zcr = zero_crossing_rate(target_signal)
    avg_freq = average_frequency(target_signal, sample_rate)  # Cần thay đổi sample_rate cho phù hợp

    # Thêm vector đại diện cho file âm thanh vào cây tìm kiếm
    feature_vector = [energy, variance, zcr, avg_freq]
    return feature_vector

In [7]:
database_path = "./dataset/"
index_path = './index.ann'  # Đường dẫn lưu trữ cây tìm kiếm
k = 5  # Số lượng láng giềng cần tìm


# Tạo cấu trúc cây tìm kiếm với Annoy
feature_dim = 4   # Số chiều của vector đặc trưng (số thuộc tính sử dụng)
index = AnnoyIndex(feature_dim, metric='euclidean')

features = []
songs = []
for song in tqdm(os.listdir(database_path)):
    song = os.path.join(database_path, song)
    y, sr = librosa.load(song, sr=8000)
    feat = extract_features(y)
    features.append(feat)
    songs.append(song)

pickle.dump(features, open('features.pk', 'wb'))

pickle.dump(songs, open('songs.pk', 'wb'))

# Thêm các vector đại diện cho các file âm thanh vào cây tìm kiếm
for i in range(len(features)):
# Thêm vector đại diện cho file âm thanh vào cây tìm kiếm
    v = features[i]
    index.add_item(i, v)

index.build(100) # 100 trees
index.save('music.ann')

100%|██████████| 100/100 [04:28<00:00,  2.68s/it]


True

In [8]:
from urllib.parse import urljoin
from urllib.request import pathname2url

def path2url(path):
    return urljoin('file:',"Users/leduo/Desktop/python/"+pathname2url(path))

song = os.path.join("./New folder", 'audio100-test.wav')
y, sr =  librosa.load(song,sr=8000)
featTest = extract_features(y)
neighbor_ids = index.get_nns_by_vector(featTest, k)
similar_files = []
for neighbor_id in neighbor_ids:
    similar_files.append(path2url(database_path+os.listdir(database_path)[neighbor_id]))

print("Most similar files:\n", "\n".join(similar_files))

Most similar files:
 file:///Users/leduo/Desktop/python/dataset/audio100%20%28mp3cut.net%29.wav
file:///Users/leduo/Desktop/python/dataset/audio28%20%28mp3cut.net%29.wav
file:///Users/leduo/Desktop/python/dataset/audio37%20%28mp3cut.net%29.wav
file:///Users/leduo/Desktop/python/dataset/audio21%20%28mp3cut.net%29.wav
file:///Users/leduo/Desktop/python/dataset/audio82%20%28mp3cut.net%29.wav
